In [ ]:
load("~/notebook/scRNA.image.RData")

In [ ]:
dir.create("/lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate//Intermediate_data/230813_gene_peak_linkage_analysis_example")

In [ ]:
setwd("/lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate//Intermediate_data/230813_gene_peak_linkage_analysis_example")

In [ ]:
gene_peak_linkage_analysis <- function(gene.id, distance, df_anno, MM, df_promoter_ATAC, df_peak_ATAC) {

        # find the adjacent peak id
        gene_location = df_promoter_ATAC %>% filter(grepl(gene.id,gene))
        find_gene = FALSE
        find_promoter = FALSE
        find_distal = FALSE
        expr.vector = 0
        promoter.vector = 0
        distal.vector = 0
        distal.num = 0
        promoter.num = 0
        distal.num = 0 
        gene.id = gene.id
        expr.cor.promoter = NULL 
        expr.cor.distal = NULL
        dis.peaks.distance = NULL

        if((nrow(gene_location) >= 1) & (gene.id %in% (df_anno$id)))
            {
            expr.vector = MM[gene.id,]
            find_gene = TRUE
            chr.site = c(gene_location$chr)[1]
            start.site = as.numeric(gene_location$start)
            end.site = as.numeric(gene_location$end)
            promoter_peak.site = unique(c(gene_location$peak))
            
#             cat("\n gene_id: ", gene.id)
#             cat("\n start_thresh: ", min(start.site) - distance)
#             cat("\n end_thresh: ", max(start.site) + distance)

            dis.peaks = (df_peak_ATAC %>% filter(chr == chr.site) 
                         %>% filter(end > min(start.site) - distance) 
                         %>% filter(start < max(start.site) + distance)
                         %>% filter(peak %in% df_anno$id)
                         %>% filter(!(peak %in% promoter_peak.site)))
            dis.peaks$promoter_dist = abs((dis.peaks$start + dis.peaks$end)/2 - mean(start.site))
#             cat("\n Number of distal site: ", nrow(dis.peaks))
#             cat("Promoter peak site:", promoter_peak.site)

            if(sum(promoter_peak.site %in% df_anno$id) > 0) {
                find_promoter = TRUE
#                 cat(sum(promoter_peak.site %in% df_anno$id),"\n")
                if(sum(promoter_peak.site %in% df_anno$id) > 1) {
                    promoter_peak.site = promoter_peak.site[promoter_peak.site %in% df_anno$id]
#                     cat(promoter_peak.site,"\n")
                    promoter.vector = t(MM[promoter_peak.site,])
                    promoter.num = ncol(promoter.vector)
#                     cat(promoter.num,"\n")
                    expr.cor.promoter = cor(promoter.vector, expr.vector)[,1]

                    }
                if(sum(promoter_peak.site %in% df_anno$id) == 1){
                    promoter_peak.site = promoter_peak.site[promoter_peak.site %in% df_anno$id]
#                     cat(promoter_peak.site,"\n")
                    promoter.vector = MM[promoter_peak.site,]
                    promoter.num = 1
                    expr.cor.promoter = cor(promoter.vector, expr.vector)
                    names(expr.cor.promoter) = promoter_peak.site
                }
            }

            if(nrow(dis.peaks) > 1) {
                find_distal = TRUE
                # filter the postively linked sites
                distal.vector = t(MM[dis.peaks$peak,])
                distal.num = ncol(distal.vector)
#                 cat("\n number of distal vector: ", distal.num)
                expr.cor.distal = cor(distal.vector, expr.vector)[, 1]
                dis.peaks.distance = (dis.peaks$start + dis.peaks$end)/2 - mean(start.site)
                names(dis.peaks.distance) = dis.peaks$peak
            }
            if(nrow(dis.peaks)==1){
                find_distal = TRUE
                # filter the postively linked sites
                distal.vector = MM[dis.peaks$peak,]
                distal.num = 1
#                 cat("\n number of distal vector: ", distal.num)
                expr.cor.distal = cor(distal.vector, expr.vector)
                names(expr.cor.distal) = dis.peaks$peak
                dis.peaks.distance = (dis.peaks$start + dis.peaks$end)/2 - mean(start.site)
                names(dis.peaks.distance) = dis.peaks$peak
                
                
                
            }
        }

        return(list(find_gene, find_promoter, find_distal, expr.vector, promoter.vector, 
                    distal.vector,promoter.num,distal.num,gene.id,
                    expr.cor.promoter,expr.cor.distal,dis.peaks.distance))
    }

In [ ]:
linkage_results_summary = function(linkage_output){
find.genes = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[1]]}))
find.promoters = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[2]]}))
find.distals = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[3]]}))
promoter.vector.ls = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[5]]}))
promoter.nums = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[7]]}))
dist.nums = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[8]]}))
gene_id_tested = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[9]]}))
pearsonr_promoter = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[10]]}))
pearsonr_dist = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[11]]}))
distal.distance = (sapply(1:length(linkage_output), function(x) {linkage_output[[x]][[12]]}))

names(distal.distance) = gene_id_tested
names(find.genes) = gene_id_tested
names(pearsonr_dist) = gene_id_tested
names(pearsonr_promoter) = gene_id_tested

df_distal_distance = data.frame(unlist(distal.distance))%>% dplyr::rename(distance = `unlist.distal.distance.`)
df_pearsonr_dist = as.data.frame(unlist(pearsonr_dist)) %>% dplyr::rename(pearsonr = `unlist(pearsonr_dist)`) %>% mutate(region='distal')

rownames_split = str_split_fixed(rownames(df_distal_distance),"[.]",3)
df_distal_distance$gene_id = paste(rownames_split[,1],rownames_split[,2],sep='.')
df_distal_distance$peak = rownames_split[,3]

df_pearsonr_promoter = as.data.frame(unlist(pearsonr_promoter)) %>% dplyr::rename(pearsonr = `unlist(pearsonr_promoter)`) %>% mutate(region='promoter')
df_pearsonr = rbind(df_pearsonr_dist,df_pearsonr_promoter)
rownames_split = str_split_fixed(rownames(df_pearsonr),"[.]",3)
df_pearsonr$gene_id = paste(rownames_split[,1],rownames_split[,2],sep='.')
df_pearsonr$peak = rownames_split[,3]
df_pearsonr = df_pearsonr %>% left_join(df_distal_distance %>% select(gene_id,peak,distance))
return(df_pearsonr)
}

# 1. generate pseudo multi-omics cells based on kmeans clustering

In [ ]:
library(ggplot2)
library(tidyverse)
library(RColorBrewer)

In [ ]:
##Read in a cell metadata table after running integration analysis, contains the umap coordinates in the column "UMAP_1" and "UMAP_2".
df_cell = read.csv("df_cell_integrated.csv",row.names=1)

In [ ]:
g=plot_categorical_label(df_cell,
                         'UMAP_1','UMAP_2','Main_cluster_name',label.text = F,
                         fig.width = 15,fig.height = 5,size=0.1)
g+facet_wrap(~assay)

In [ ]:
##First, define the mean number of cells per kmeans cluster you want, and perform kmeans clustering using function "kmeans()"
mean_num_per_cluster=150
n = max(2, ceiling(nrow(df_cell) / mean_num_per_cluster))
kmeans_clusters <- kmeans(df_cell %>% select(UMAP_1, UMAP_2), centers = n, 
                          nstart = 20, iter.max = 20)
df_cell$cluster = as.character(kmeans_clusters$cluster)

In [ ]:
g=plot_categorical_label(df_cell,
                         'UMAP_1','UMAP_2','cluster',label.text = T,
                         fig.width = 5,fig.height = 5,size=0.1)
g

# 2. Aggregate RNA and ATAC cells from the same kmeans cluster

In [ ]:
cds_RNA = readRDS("//lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate//Intermediate_data/230813_gene_peak_linkage_analysis_example/cds_RNA.rds")
cds_ATAC = readRDS("/lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate//Intermediate_data/230813_gene_peak_linkage_analysis_example/cds_ATAC.rds")

In [ ]:
cds.RNA$cluster = as.numeric(df_cell[cds.RNA$sample,'cluster'])
cds.ATAC$cluster = as.numeric(df_cell[cds.ATAC$sample,'cluster'])

In [ ]:
n = length(unique(df_cell$cluster))
df_celltype_ratio_kmeans = as.matrix(table(df_cell$cluster,df_cell$Main_cluster_name))/as.vector(table(df_cell$cluster))
id = colnames(df_celltype_ratio_kmeans)[apply(df_celltype_ratio_kmeans,1,which.max)]
agg_cellid=paste(1:n, id, sep = "_")

agg_gene_count = lapply(1:n, function(x) {
        sample_name_list = (pData(cds.RNA)[cds.RNA$cluster == x, ])$sample
        return(Matrix::rowSums((exprs(cds.RNA))[, cds.RNA$sample %in% sample_name_list]))
        })

agg_peak_count = lapply(1:n, function(x) {
        sample_name_list = (pData(cds.ATAC)[cds.ATAC$cluster == x, ])$sample
        return(Matrix::rowSums((exprs(cds.ATAC))[, cds.ATAC$sample %in% sample_name_list]))
        })
agg_gene_count = do.call(cbind, agg_gene_count)
agg_peak_count = do.call(cbind, agg_peak_count)
    
colnames(agg_gene_count) = agg_cellid
colnames(agg_peak_count) = agg_cellid

In [ ]:
head(agg_gene_count)[,1:10]

In [ ]:
head(agg_peak_count)[,1:10]

In [ ]:
df_cell_RNA_cluster = data.frame(cds.RNA$cluster)
df_cell_ATAC_cluster = data.frame(cds.ATAC$cluster)
df_cell_name_RNA = data.frame("cell_name" = colnames(agg_gene_count), 
                             "cell_num" = sapply(1:n, function(x) {return(sum(df_cell_RNA_cluster[, 1] == x))}))
df_cell_name_ATAC = data.frame("cell_name" = colnames(agg_peak_count), 
                             "cell_num" = sapply(1:n, function(x) {return(sum(df_cell_ATAC_cluster[, 1] == x))}))
df_cell_name_RNA$Main_cluster_name = str_split_fixed(df_cell_name_RNA$cell_name,'_',2)[,2]
df_cell_name_ATAC$Main_cluster_name = str_split_fixed(df_cell_name_ATAC$cell_name,'_',2)[,2]
rownames(df_cell_name_RNA) = df_cell_name_RNA$cell_name
rownames(df_cell_name_ATAC) = df_cell_name_ATAC$cell_name

In [ ]:
rownames(df_cell_name_RNA) = make.names(rownames(df_cell_name_RNA))
df_cell_name_RNA$sample = rownames(df_cell_name_RNA)
cds_agg_RNA = cds_construct(agg_gene_count, df_cell_name_RNA, fData(cds.RNA))
rownames(df_cell_name_ATAC) = make.names(rownames(df_cell_name_ATAC))
df_cell_name_ATAC$sample = rownames(df_cell_name_ATAC)
cds_agg_ATAC = cds_construct(agg_peak_count, df_cell_name_ATAC, fData(cds.ATAC)%>%mutate(gene_id=peak))

In [ ]:
saveRDS(cds_agg_RNA,"//lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate//Intermediate_data/230813_gene_peak_linkage_analysis_example/cds_agg_RNA.rds")
saveRDS(cds_agg_ATAC,"//lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate//Intermediate_data/230813_gene_peak_linkage_analysis_example/cds_agg_ATAC.rds")

# 3. perform linkage analysis

In [ ]:
library(tidyverse)
library(Matrix.utils)
library(Matrix)
library(stringr)
library(DESeq2)
library(cowplot)
library(BiocParallel)
library(monocle)

In [ ]:
cds_agg_RNA = readRDS("/lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate/Intermediate_data/ZL_20220218_sciRate_RNA_ATAC_linkage_analysis/cds_agg_RNA.rds")
cds_agg_ATAC = readRDS("/lustre/fs4/cao_lab/scratch/zlu/ZL_20210113_sciRate/Intermediate_data/ZL_20220218_sciRate_RNA_ATAC_linkage_analysis/cds_agg_ATAC.rds")

In [ ]:
agg_peak_count = exprs(cds_agg_ATAC)
agg_gene_count = exprs(cds_agg_RNA)
df_peak_ATAC = fData(cds_agg_ATAC)

In [ ]:
df_promoter_ATAC = read.csv('df_promoter_ATAC.csv',row.names = 1)

In [ ]:
head(df_promoter_ATAC)

In [ ]:
TPM_RNA = agg_gene_count/colSums(agg_gene_count)*1e+6
TPM_ATAC = agg_peak_count/colSums(agg_peak_count)*1e+6
max_TPM_RNA = apply(TPM_RNA,1,max)
max_TPM_ATAC = apply(TPM_ATAC,1,max)
TPM_RNA_fil = TPM_RNA[max_TPM_RNA>10,]
TPM_ATAC_fil = TPM_ATAC[max_TPM_ATAC>10,]

In [ ]:
gene = data.frame("type" = "gene", "id" = row.names(TPM_RNA_fil))
peak = data.frame("type" = "peak", "id" = row.names(TPM_ATAC_fil))
MM = rbind(log2(TPM_RNA_fil+0.01), log2(TPM_ATAC_fil+0.01))
df_anno = rbind(gene, peak)

In [ ]:
TPM_ATAC_fil_permute = TPM_ATAC_fil[,sample(1:ncol(TPM_ATAC_fil),ncol(TPM_ATAC_fil))]
colnames(TPM_ATAC_fil_permute) = colnames(TPM_ATAC_fil)
MM_permute = rbind(log2(TPM_RNA_fil+0.01), log2(TPM_ATAC_fil_permute+0.01))

In [ ]:
gene_list = rownames(TPM_RNA_fil)
distance = 500000

linkage_output = lapply(gene_list, function(x) {gene_peak_linkage_analysis(x, distance, df_anno, MM, df_promoter_ATAC, df_peak_ATAC)})
df_pearsonr = linkage_results_summary(linkage_output)

In [ ]:
linkage_output_permute = lapply(gene_list, function(x) {gene_peak_linkage_analysis(x, distance, df_anno, MM_permute, df_promoter_ATAC, df_peak_ATAC)})
df_pearsonr_permute = linkage_results_summary(linkage_output_permute)

In [ ]:
df_pearsonr_permute$permutation = 'cell_id_permutated'
df_pearsonr$permutation = 'no_permutation'

In [ ]:
df_pearsonr_combine_positive = rbind(df_pearsonr %>% select(pearsonr,permutation),df_pearsonr_permute %>% select(pearsonr,permutation)) %>% filter(pearsonr>0)

In [ ]:
FDR_cutoff=0.05
cutoff = seq(0.15,0.5,0.0025)
FDR = list()
True_link_num = list()
False_link_num= list()

for(pearsonr_cut in cutoff){
df_pearsonr_combine_positive$pass_cutoff = df_pearsonr_combine_positive$pearsonr>pearsonr_cut
df=table(df_pearsonr_combine_positive$pass_cutoff,df_pearsonr_combine_positive$permutation)
tmp_FDR=df[2,1]/sum(df[2,1]+df[2,2])
# print(paste0(pearsonr_cut,":",tmp_FDR))
FDR[as.character(pearsonr_cut)] = tmp_FDR
True_link_num[as.character(pearsonr_cut)] = df[2,2]
False_link_num[as.character(pearsonr_cut)] = df[2,1]
}
df = as.data.frame(cbind(unlist(FDR),
      unlist(True_link_num),
      unlist(False_link_num)))
colnames(df) = c("FDR","Link","Link_permutated")
df$pearsonr_cutoff = as.numeric(rownames(df))

pearsonr_cutoff = min((df %>% filter(FDR<=FDR_cutoff))$pearsonr_cutoff)

In [ ]:
df_pearsonr_filtered = df_pearsonr %>% dplyr::filter(pearsonr >= pearsonr_cutoff)

In [ ]:
saveRDS(df_pearsonr_filtered,"df_pearsonr_filtered.rds")